Before you start, make sure you have the following packages installed and set up. Note that you should have the same version specified in the requirements.txt.

The package segment_anything is a bit different when setting up. You should set it up following the instructions here: https://github.com/facebookresearch/segment-anything/tree/main (in the installation part)

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from segment_anything import sam_model_registry, SamPredictor
from utils import *
import os

To DO:
1. Before running the following block, make sure your desired photos are uploaded into the folder 'Images', with the naming convention 'xxx_1.jpg'and 'xxx_2.jpg'. We consider the 'xxx_1.jpg' as the cameraman image, and 'xxx_2.jpg' as the group photo.
(Note that, if you want to change the input images to a different style type, change the img_suffix to your desired type as well, for example '.jpeg', or '.png')
2. Change the xxx to the name of your desired photos

In [ ]:
img_path = 'Images'
img_name = 'xxx'
img_suffix = '.jpg'
os.makedirs('result/' + img_name, exist_ok=True)

In [ ]:
img1 = cv2.imread(img_path + '/' + img_name + '_1' + img_suffix)
img2 = cv2.imread(img_path + '/' + img_name + '_2' + img_suffix)
img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
_, axs = plt.subplots(1, 2, figsize=(20, 10))
axs[0].imshow(img1)
axs[0].set_axis_off()
axs[1].imshow(img2)
axs[1].set_axis_off()
plt.show()

In [ ]:
H = findHomography(img1, img2, showMatches=True, filePath='result/' + img_name)
_, axs = plt.subplots(2, 2)
axs[0, 0].imshow(img1)
axs[0, 0].set_axis_off()
axs[0, 0].set_title("Original Image 1")
axs[0, 1].imshow(img2)
axs[0, 1].set_axis_off()
axs[0, 1].set_title("Original Image 2")
axs[1, 0].imshow(cv2.warpPerspective(img1, H, (img1.shape[1], img1.shape[0])))
axs[1, 0].set_axis_off()
axs[1, 0].set_title("Warped Image 1")
plt.axis('off')
plt.savefig(f'result/{img_name}/homography.jpg')

In [ ]:
warped_img1 = cv2.warpPerspective(img1, H, (img1.shape[1], img1.shape[0]))
cv2.imwrite(f'result/{img_name}/warped_img1.jpg', cv2.cvtColor(warped_img1, cv2.COLOR_RGB2BGR))
plt.imshow(warped_img1)
plt.axis('off')

To DO:


For the following block, if you are running the notebook on some new photos, run as it is. (i.e.run the portion of block denoted as 1). If you have already found the mask for the photo and you want to use the saved mask, comment out portion 1 and uncomment portion 2. (You can still run as default, but running portion 2 if a mask already existed is faster.)

In [ ]:
bbox = findBoundingBox(warped_img1, winStride=(10, 10), scale=1.05)
print(bbox)
# 1
mask = findMask(warped_img1, bbox)
mask = mask[0]
np.save('tmp/mask.npy', mask[0])
np.save('tmp/bbox.npy', bbox)

# 2
# mask = np.load('tmp/mask.npy')
# bbox = np.load('tmp/bbox.npy')

plt.figure(figsize=(10, 10))
plt.imshow(warped_img1)
plt_mask(mask, plt.gca())
plt_bbox(bbox, plt.gca())
plt.savefig(f'result/{img_name}/bbox&mask.jpg')
plt.show()

In [ ]:
new_img = maskTransfer(warped_img1, img2, mask)
cv2.imwrite(f'result/{img_name}/new_img.jpg', cv2.cvtColor(new_img, cv2.COLOR_RGB2BGR))
plt.imshow(new_img)
plt.axis('off')
plt.show()   

To Do:

After running the above cell, you will find the resulting images in the folder 'result'. The above approach is our primary approach for this project. If you want to try some other approaches and compare the differences, you are free to run the following cells to see the different outputs from various approaches. However, in our minds, the above approach produces the best results. Please find the discussion part in our final report for more references on this matter. 

In [ ]:
# Box Transfer approach
new_img2 = bboxTransfer(warped_img1, img2, bbox)
cv2.imwrite(f'result/{img_name}/new_img2.jpg', cv2.cvtColor(new_img2, cv2.COLOR_RGB2BGR))
plt.imshow(new_img2)
plt.axis('off')
plt.show()

In [ ]:
# Poisson Blending approach
new_img3 = poissonBlending(warped_img1, img2, mask)
cv2.imwrite(f'result/{img_name}/new_img3.jpg', cv2.cvtColor(new_img3, cv2.COLOR_RGB2BGR))
plt.imshow(new_img3)
plt.axis('off')
plt.show()

In [ ]:
# Gaussian Mask Blending approach
new_img4 = gaussianMaskBlending(warped_img1, img2, mask)
cv2.imwrite(f'result/{img_name}/new_img4.jpg', cv2.cvtColor(new_img4, cv2.COLOR_RGB2BGR))
plt.imshow(new_img4)
plt.axis('off')
plt.show()